In [1]:
import igraph
import numpy as np
import pandas as pd
import pickle
import skmob
from shapely.geometry import Point
from skmob.tessellation import tilers


In [14]:
CACHE = True
filename_fdf = 'fdf.pkl'

In [3]:
import geopandas as gpd
tessellation = tilers.tiler.get("squared", base_shape="Manhattan, New York City, USA", meters=1500)

In [4]:
tessellation

,tile_ID,geometry
0,0,"POLYGON ((-74.04722 40.68394, -74.04722 40.694..."
1,1,"POLYGON ((-74.04722 40.69416, -74.04722 40.704..."
2,2,"POLYGON ((-74.03375 40.68394, -74.03375 40.694..."
3,3,"POLYGON ((-74.02027 40.68394, -74.02027 40.694..."
4,4,"POLYGON ((-74.02027 40.69416, -74.02027 40.704..."
...,...,...
74,74,"POLYGON ((-73.92595 40.86761, -73.92595 40.877..."
75,75,"POLYGON ((-73.92595 40.87780, -73.92595 40.887..."
76,76,"POLYGON ((-73.91247 40.85742, -73.91247 40.867..."
77,77,"POLYGON ((-73.91247 40.86761, -73.91247 40.877..."


In [5]:
FILENAME = "utils/2013-12 - Citi Bike trip data.csv"

In [6]:
df = pd.read_csv(FILENAME, sep=',')

In [7]:
df.head()


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1015,2013-12-01 00:00:03,2013-12-01 00:16:58,401,Allen St & Rivington St,40.720196,-73.989978,476,E 31 St & 3 Ave,40.743943,-73.979661,14729,Subscriber,1979,2
1,962,2013-12-01 00:00:07,2013-12-01 00:16:09,312,Allen St & E Houston St,40.722055,-73.989111,223,W 13 St & 7 Ave,40.737815,-73.999947,17871,Subscriber,1970,2
2,768,2013-12-01 00:00:15,2013-12-01 00:13:03,326,E 11 St & 1 Ave,40.729538,-73.984267,291,Madison St & Montgomery St,40.713126,-73.984844,18508,Subscriber,1982,2
3,218,2013-12-01 00:00:49,2013-12-01 00:04:27,2021,W 45 St & 8 Ave,40.759291,-73.988597,447,8 Ave & W 52 St,40.763707,-73.985162,17745,Subscriber,1976,1
4,550,2013-12-01 00:01:28,2013-12-01 00:10:38,439,E 4 St & 2 Ave,40.726281,-73.989780,296,Division St & Bowery,40.714131,-73.997047,16947,Subscriber,1983,1


In [8]:
df = df[["starttime","stoptime", "start station latitude", "start station longitude", "end station latitude", "end station longitude"]]

In [19]:
def getIndexPosition(tessellation, position):
    for i, row in enumerate(tessellation):
        if row.contains(position):
            return i

In [20]:
if False:
        with open(filename_fdf, "rb") as input_file:
            fdf = pickle.load(input_file)
        # fdf = fdf[["origin", "destination", "starttime", "stoptime"]]
        # fdf['flow'] = 1
else: 
    url_tess = skmob.utils.constants.NY_COUNTIES_2011
    tessellation = tilers.tiler.get("squared", base_shape="Manhattan, New York City, USA", meters=1500)

    df = pd.read_csv(FILENAME, sep=',')
    df = df[["starttime","stoptime", "start station latitude", "start station longitude", "end station latitude", "end station longitude"]]
    df['origin'] = df.apply(lambda row: Point(row['start station longitude'], row['start station latitude']), axis=1)
    df['destination'] = df.apply(lambda row: Point(row['end station longitude'], row['end station latitude']), axis=1)

    poligons = tessellation['geometry']

    # Long time operation
    df['origin'] = df.apply(lambda row: getIndexPosition(poligons, row['origin']), axis=1)
    df['destination'] = df.apply(lambda row: getIndexPosition(poligons, row['destination']), axis=1)

    # try:
    #     fdf = skmob.FlowDataFrame(df,
    #                         tessellation=tessellation,
    #                         origin="origin",
    #                         destination="destination",
    #                         tile_id='tile_ID')
    # except:
    #     pass
    # fdf = fdf[["origin", "destination", "starttime", "stoptime"]]
    # fdf['flow'] = 1
    # with open('fdf.pkl', 'wb') as f:
    #     pickle.dump(fdf, f, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('df.pkl', 'wb') as f:
        pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
df

,starttime,stoptime,start station latitude,start station longitude,end station latitude,end station longitude,origin,destination
0,2013-12-01 00:00:03,2013-12-01 00:16:58,40.720196,-73.989978,40.743943,-73.979661,20.0,31.0
1,2013-12-01 00:00:07,2013-12-01 00:16:09,40.722055,-73.989111,40.737815,-73.999947,20.0,15.0
2,2013-12-01 00:00:15,2013-12-01 00:13:03,40.729538,-73.984267,40.713126,-73.984844,21.0,19.0
3,2013-12-01 00:00:49,2013-12-01 00:04:27,40.759291,-73.988597,40.763707,-73.985162,24.0,24.0
4,2013-12-01 00:01:28,2013-12-01 00:10:38,40.726281,-73.989780,40.714131,-73.997047,21.0,12.0
...,...,...,...,...,...,...,...,...
443961,2013-12-31 23:55:56,2014-01-01 00:12:50,40.715595,-73.987030,40.710451,-73.960876,20.0,NaN
443962,2013-12-31 23:57:04,2014-01-01 00:08:26,40.757952,-73.977876,40.765909,-73.976342,33.0,34.0
443963,2013-12-31 23:57:44,2014-01-01 00:23:04,40.764397,-73.973715,40.722281,-73.976687,33.0,29.0
443964,2013-12-31 23:58:14,2014-01-01 00:17:40,40.750200,-73.990931,40.735439,-73.994539,23.0,15.0


# Proseguire da qua.
## Attualmente ho un dataframe, al quale devo aggiungere il flow e rimuovere i Nan dagli origin e destination

In [31]:
df.dropna(inplace=True)
fdf = df[["origin", "destination", "starttime", "stoptime"]]
fdf['origin'] = fdf['origin'].astype(int)
fdf['destination'] = fdf['destination'].astype(int)
fdf['flow'] = 1

In [32]:
fdf

,origin,destination,starttime,stoptime,flow
0,20,31,2013-12-01 00:00:03,2013-12-01 00:16:58,1
1,20,15,2013-12-01 00:00:07,2013-12-01 00:16:09,1
2,21,19,2013-12-01 00:00:15,2013-12-01 00:13:03,1
3,24,24,2013-12-01 00:00:49,2013-12-01 00:04:27,1
4,21,12,2013-12-01 00:01:28,2013-12-01 00:10:38,1
...,...,...,...,...,...
443960,15,15,2013-12-31 23:55:06,2013-12-31 23:59:14,1
443962,33,34,2013-12-31 23:57:04,2014-01-01 00:08:26,1
443963,33,29,2013-12-31 23:57:44,2014-01-01 00:23:04,1
443964,23,15,2013-12-31 23:58:14,2014-01-01 00:17:40,1


In [33]:
fdf = skmob.FlowDataFrame(fdf,
                        tessellation=tessellation,
                        origin="origin",
                        destination="destination",
                        tile_id='tile_ID')

In [34]:
flow_df = fdf[['starttime','origin','destination', 'flow']]

In [35]:
flow_df['starttime'] = pd.to_datetime(flow_df.starttime)

In [42]:
flow_df['origin'] = flow_df['origin'].astype(int)
flow_df['destination'] = flow_df['destination'].astype(int)

In [55]:
# flow_df['origin'] = flow_df['origin'].astype(int)-int(flow_df['origin'].min())
# flow_df['destination'] = flow_df['destination'].astype(int)-int(flow_df['destination'].min())

In [43]:
print(flow_df.min())
print(flow_df.max())

starttime      2013-12-01 00:00:03
origin                           4
destination                      4
flow                             1
dtype: object
starttime      2013-12-31 23:58:16
origin                          40
destination                     40
flow                             1
dtype: object


In [44]:
flow_df.head()

,starttime,origin,destination,flow
0,2013-12-01 00:00:03,20,31,1
1,2013-12-01 00:00:07,20,15,1
2,2013-12-01 00:00:15,21,19,1
3,2013-12-01 00:00:49,24,24,1
4,2013-12-01 00:01:28,21,12,1


In [62]:
# f_out = flow_df.groupby([pd.Grouper(key='starttime', freq='60min'),'origin']).sum()
# f_out

flow
starttime           origin      
2013-12-01 00:00:00 0          1
                    10        13
                    11        16
                    12         8
                    13         3
...                          ...
2013-12-31 23:00:00 28         2
                    29         4
                    4          1
                    8          5
                    9          4

[18170 rows x 1 columns]

In [45]:
# len(f_out)

NameError: name 'f_out' is not defined

In [73]:
# m = f_out.plot_tessellation(popup_features=['tile_ID', 'flow'])
# m

In [135]:
# x_list

[-74.0404845353691,
 -74.01353507684551,
 -74.04048453536909,
 -74.0270098061073,
 -74.00006034758371,
 -73.94616143053655,
 -73.93268670127475,
 -73.91921197201296,
 -73.90573724275117,
 -73.90573724275116,
 -74.0135350768455,
 -73.93268670127476,
 -73.91921197201295,
 -73.93268670127473,
 -73.91921197201297,
 -73.98658561832193,
 -73.97311088906014,
 -73.97311088906012,
 -73.95963615979832,
 -73.95963615979834,
 -73.95963615979835]

In [46]:
tessellation['centroids'] = [el.centroid for el in tessellation['geometry']]

In [47]:
x_list = []
y_list = []
for el in tessellation['geometry']:
    x_list.append(el.centroid.x)
    y_list.append(el.centroid.y)
x_list = sorted(list(set(x_list)))
y_list = sorted(list(set(y_list)))
print(len(x_list))
print(len(y_list))

21
28


In [48]:
x_dict = {key:val for val, key in enumerate(x_list)}
y_dict = {key:val for val, key in enumerate(y_list)}

In [49]:
tessellation['map_point'] = [(x_dict[centroid.x], y_dict[centroid.y]) for centroid in tessellation['centroids']]

In [50]:
tile_to_xy = {key:val for (key,val) in zip(tessellation['tile_ID'], tessellation['map_point'])}

In [55]:
flow_df['origin'] = [tile_to_xy[str(el)] for el in flow_df['origin']]
flow_df['destination'] = [tile_to_xy[str(el)] for el in flow_df['destination']]

In [56]:
f_out = flow_df.groupby([pd.Grouper(key='starttime', freq='60min'),'origin']).sum()
f_out

flow
starttime           origin       
2013-12-01 00:00:00 (3, 2)      1
                    (3, 6)      3
                    (3, 12)     2
                    (4, 9)      3
                    (5, 3)      1
...                           ...
2013-12-31 23:00:00 (7, 6)      2
                    (7, 7)      3
                    (7, 9)     10
                    (7, 13)     4
                    (8, 12)     2

[18170 rows x 1 columns]

In [207]:
f_in = flow_df.groupby([pd.Grouper(key='starttime', freq='60min'),'destination']).sum()
f_in

flow
starttime           destination      
2013-12-01 00:00:00 (3, 4)          3
                    (3, 6)          1
                    (3, 7)          4
                    (4, 9)          1
                    (5, 3)          2
...                               ...
2013-12-31 23:00:00 (7, 9)          4
                    (7, 13)         4
                    (7, 14)         2
                    (8, 12)         7
                    (10, 13)        1

[18406 rows x 1 columns]

In [212]:
n_timestamps = flow_df.groupby(pd.Grouper(key='starttime', freq='60min')).ngroups

In [213]:
flows = 2
# Riempimento numpy array 
X_dataset = np.zeros([n_timestamps, len(x_list), len(y_list), flows])

# Outflow
time_samples = set()
t = -1
for metadata, flow in f_out.iterrows():
    if metadata[0] not in time_samples:
        time_samples.add(metadata[0])
        t += 1
    x = metadata[1][0]
    y = metadata[1][1]
    X_dataset[t][int(x)][int(y)][0] = flow['flow']

# Inflow
time_samples = set()
t = -1
for metadata, flow in f_in.iterrows():
    if metadata[0] not in time_samples:
        time_samples.add(metadata[0])
        t += 1
    x = metadata[1][0]
    y = metadata[1][1]
    X_dataset[t][int(x)][int(y)][1] = flow['flow']

In [197]:
f_out

flow
starttime           origin       
2013-12-01 00:00:00 (3, 2)      1
                    (3, 6)      3
                    (3, 12)     3
                    (4, 9)      3
                    (5, 3)      1
...                           ...
2013-12-31 23:00:00 (7, 6)      2
                    (7, 7)      3
                    (7, 9)     10
                    (7, 13)     4
                    (8, 12)     2

[18170 rows x 1 columns]

In [214]:
time_samples = list(time_samples)

In [215]:
np.shape(X_dataset)

(744, 21, 28, 2)

In [216]:
T = 24
PeriodInterval = 1
TrendInterval = 7
len_closeness = 3  # length of closeness dependent sequence
len_period = 1  # length of peroid dependent sequence
len_trend = 1  # length of trend dependent sequence
c_list = range(1, len_closeness+1)
p_list = [PeriodInterval * T * j for j in range(1, len_period+1)]
t_list = [TrendInterval * T * j for j in range(1, len_trend+1)]

In [115]:
from utils.load_datasets.STMatrix import STMatrix

In [113]:
st_matrix = STMatrix()

[168]

In [122]:
from utils.config import Config
import h5py

In [120]:
# parameter
DATAPATH = Config().DATAPATH
def load_stdata(fname):
    f = h5py.File(fname, 'r')
    data = f['data'].value
    timestamps = f['date'].value
    f.close()
    return data, timestamps

In [158]:
#  load data
data, timestamps = load_stdata(os.path.join(DATAPATH, 'BikeNYC', 'NYC14_M16x8_T60_NewEnd.h5'))
print((timestamps))
print(timestamps)
print(np.shape(data))

[b'2014040101' b'2014040102' b'2014040103' ... b'2014093022' b'2014093023'
 b'2014093024']
[b'2014040101' b'2014040102' b'2014040103' ... b'2014093022' b'2014093023'
 b'2014093024']
(4392, 2, 16, 8)


In [147]:
from utils.load_datasets import BikeNYC

In [178]:
nb_flow = 2
len_test = 10

In [231]:
from __future__ import print_function
import os
import pickle
import numpy as np

# from . import load_stdata
from utils.minmax_normalization import MinMaxNormalization
from utils.preprocessing_functions import remove_incomplete_days, timestamp2vec
from utils.config import Config
from utils.load_datasets.STMatrix import STMatrix
np.random.seed(1337)  # for reproducibility

# parameters
DATAPATH = Config().DATAPATH


def load_data(T=24, nb_flow=2, len_closeness=None, len_period=None, len_trend=None, len_test=None, preprocess_name='preprocessing.pkl', meta_data=True, stdata_file=None):
    assert(len_closeness + len_period + len_trend > 0)
    # load data
    if stdata_file is None:
        data, timestamps = load_stdata(os.path.join(DATAPATH, 'BikeNYC', 'NYC14_M16x8_T60_NewEnd.h5'))
    else:
        data, timestamps = stdata_file
    # print(timestamps)
    # remove a certain day which does not have 24 timestamps
    # data, timestamps = remove_incomplete_days(data, timestamps, T)
    data = data[:, :, :, :nb_flow]
    data[data < 0] = 0.

    # Reorder data to have CWH
    # data = np.transpose(data, (0, 2, 3, 1))

    data_all = [data]
    timestamps_all = [timestamps]
    # minmax_scale
    data_train = data[:-len_test]
    print('train_data shape: ', data_train.shape)
    mmn = MinMaxNormalization()
    mmn.fit(data_train)
    data_all_mmn = []
    for d in data_all:
        data_all_mmn.append(mmn.transform(d))

    fpkl = open(preprocess_name, 'wb')
    for obj in [mmn]:
        pickle.dump(obj, fpkl)
    fpkl.close()

    XC, XP, XT = [], [], []
    Y = []
    timestamps_Y = []
    for data, timestamps in zip(data_all_mmn, timestamps_all):
        # instance-based dataset --> sequences with format as (X, Y) where X is a sequence of images and Y is an image.
        print(timestamps[0])
        st = STMatrix(data, timestamps, T, CheckComplete=False)
        _XC, _XP, _XT, _Y, _timestamps_Y = st.create_dataset(len_closeness=len_closeness, len_period=len_period, len_trend=len_trend)
        XC.append(_XC)
        XP.append(_XP)
        XT.append(_XT)
        Y.append(_Y)
        timestamps_Y += _timestamps_Y

    XC = np.vstack(XC)
    XP = np.vstack(XP)
    XT = np.vstack(XT)
    Y = np.vstack(Y)
    print("XC shape: ", XC.shape, "XP shape: ", XP.shape, "XT shape: ", XT.shape, "Y shape:", Y.shape)
    XC_train, XP_train, XT_train, Y_train = XC[:-len_test], XP[:-len_test], XT[:-len_test], Y[:-len_test]
    XC_test, XP_test, XT_test, Y_test = XC[-len_test:], XP[-len_test:], XT[-len_test:], Y[-len_test:]
    
    timestamp_train, timestamp_test = timestamps_Y[:-len_test], timestamps_Y[-len_test:]
    X_train = []
    X_test = []
    for l, X_ in zip([len_closeness, len_period, len_trend], [XC_train, XP_train, XT_train]):
        if l > 0:
            X_train.append(X_)
    for l, X_ in zip([len_closeness, len_period, len_trend], [XC_test, XP_test, XT_test]):
        if l > 0:
            X_test.append(X_)
    print('train shape:', XC_train.shape, Y_train.shape, 'test shape: ', XC_test.shape, Y_test.shape)
    # load meta feature
    if meta_data:
        meta_feature = timestamp2vec(timestamps_Y)
        metadata_dim = meta_feature.shape[1]
        meta_feature_train, meta_feature_test = meta_feature[:-len_test], meta_feature[-len_test:]
        X_train.append(meta_feature_train)
        X_test.append(meta_feature_test)
    else:
        metadata_dim = None
    for _X in X_train:
        print(_X.shape, )
    print()
    for _X in X_test:
        print(_X.shape, )
    print()
    return X_train, Y_train, X_test, Y_test, mmn, metadata_dim, timestamp_train, timestamp_test


In [227]:
str(time_samples[0]).replace("-", "").replace(" ","").replace(":", "")[:10]

'2013122805'

In [219]:
len(time_samples)

744

In [220]:
len([str(time_sample).replace("-", "").replace(" ","").replace(":", "")[:10] for time_sample in time_samples])

744

In [248]:
time_string = [str(int(str(time_sample).replace("-", "").replace(" ","").replace(":", "")[:10])+1).encode('utf-8') for time_sample in time_samples]

In [249]:
X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = load_data(
            T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
            preprocess_name='my_preprocessing.pkl', meta_data=True, 
            # stdata_file=(X_dataset, [str(time_sample).replace("-", "").replace(" ","").replace(":", "")[:12] for time_sample in time_samples]))
            stdata_file= (X_dataset, time_string))

train_data shape:  (734, 21, 28, 2)
min: 0.0 max: 296.0
b'2013122806'
XC shape:  (455, 21, 28, 6) XP shape:  (455, 21, 28, 2) XT shape:  (455, 21, 28, 2) Y shape: (455, 21, 28, 2)
XC shape:  (455, 21, 28, 6) XP shape:  (455, 21, 28, 2) XT shape:  (455, 21, 28, 2) Y shape: (455, 21, 28, 2)
train shape: (445, 21, 28, 6) (445, 21, 28, 2) test shape:  (10, 21, 28, 6) (10, 21, 28, 2)
(445, 21, 28, 6)
(445, 21, 28, 2)
(445, 21, 28, 2)
(445, 8)

(10, 21, 28, 6)
(10, 21, 28, 2)
(10, 21, 28, 2)
(10, 8)



# Altra roba per la creazione del grafo

In [17]:
time = set()
time_orgin_dest = []
origin_dest = np.zeros(
            [int(flow_df['origin'].max())+1-int(flow_df['origin'].min()), 
            int(flow_df['destination'].max())+1-int(flow_df['destination'].min())])
flow = list(f['flow'])

for i, el in enumerate(f.index):
    if(el[0] not in time):
        time.add(el[0])
        time_orgin_dest.append(origin_dest)
        origin_dest = np.zeros(
            [int(flow_df['origin'].max())+1-int(flow_df['origin'].min()), 
            int(flow_df['destination'].max())+1-int(flow_df['destination'].min())])
    origin_dest[el[1],el[2]] = flow[i]

In [18]:
np.shape(time_orgin_dest)

(744, 145, 145)

In [15]:
from utils.preprocessing_tessellation import generate_inflow_outflow_map

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 15, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [16]:
f_inflow, f_outflow = generate_inflow_outflow_map(flow_df)

In [17]:
f_inflow

starttime            destination
2013-12-01 00:00:00  24             2
                     25             3
                     26             1
                     27             6
                     49             2
                                   ..
2013-12-31 23:30:00  83             4
                     105            2
                     109            1
                     113            1
                     139            1
Name: flow, Length: 40641, dtype: int64

In [22]:
tdf = tdf.rename(columns = {"start station latitude": "lat", "start station longitude": "lng", "starttime":"datetime", "bikeid":"uid"})

In [23]:
tdf

,tripduration,datetime,stoptime,start station id,start station name,lat,lng,end station id,end station name,end station latitude,end station longitude,uid,usertype,birth year,gender
0,1015,2013-12-01 00:00:03,2013-12-01 00:16:58,401,Allen St & Rivington St,40.720196,-73.989978,476,E 31 St & 3 Ave,40.743943,-73.979661,14729,Subscriber,1979,2
1,962,2013-12-01 00:00:07,2013-12-01 00:16:09,312,Allen St & E Houston St,40.722055,-73.989111,223,W 13 St & 7 Ave,40.737815,-73.999947,17871,Subscriber,1970,2
2,768,2013-12-01 00:00:15,2013-12-01 00:13:03,326,E 11 St & 1 Ave,40.729538,-73.984267,291,Madison St & Montgomery St,40.713126,-73.984844,18508,Subscriber,1982,2
3,218,2013-12-01 00:00:49,2013-12-01 00:04:27,2021,W 45 St & 8 Ave,40.759291,-73.988597,447,8 Ave & W 52 St,40.763707,-73.985162,17745,Subscriber,1976,1
4,550,2013-12-01 00:01:28,2013-12-01 00:10:38,439,E 4 St & 2 Ave,40.726281,-73.989780,296,Division St & Bowery,40.714131,-73.997047,16947,Subscriber,1983,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443961,1014,2013-12-31 23:55:56,2014-01-01 00:12:50,350,Clinton St & Grand St,40.715595,-73.987030,532,S 5 Pl & S 4 St,40.710451,-73.960876,16336,Subscriber,1973,1
443962,682,2013-12-31 23:57:04,2014-01-01 00:08:26,538,W 49 St & 5 Ave,40.757952,-73.977876,2006,Central Park S & 6 Ave,40.765909,-73.976342,18352,Subscriber,1986,1
443963,1520,2013-12-31 23:57:44,2014-01-01 00:23:04,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,411,E 6 St & Avenue D,40.722281,-73.976687,15000,Subscriber,1985,1
443964,1166,2013-12-31 23:58:14,2014-01-01 00:17:40,492,W 33 St & 7 Ave,40.750200,-73.990931,253,W 13 St & 5 Ave,40.735439,-73.994539,21022,Subscriber,1955,1


In [104]:
tdf_tid = tdf.mapping(tessellation, remove_na=True)
relevances = tdf_tid.groupby(by='tile_ID').count()[['lat']].rename(columns={'lat': 'relevance'})
# normalise
relevances /= relevances.sum()

tessellation = tessellation.merge(relevances, right_index=True, left_on='tile_ID', how='left').fillna(0.)
tessellation.head(3)

,tile_ID,geometry,relevance
0,0,"POLYGON ((-74.04722 40.68394, -74.04722 40.694...",0.0
1,1,"POLYGON ((-74.04722 40.69416, -74.04722 40.704...",0.0
2,2,"POLYGON ((-74.03375 40.68394, -74.03375 40.694...",0.0


In [106]:
fdf_train.plot_flows(min_flow=5, zoom=10, tiles='cartodbpositron', flow_weight=2, opacity=0.25)